In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd
#import requests
from bs4 import BeautifulSoup
import time
import re

from pathlib import Path
import time

In [2]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

In [13]:
# 검색 함수
def Crawl(k):
    # k <- keyword
    k = k.replace(' ','+')
    browser = webdriver.Chrome(service=service,options=chrome_options)
    browser.maximize_window()
    url = f"https://search.naver.com/search.naver?query={k}&nso=&where=blog&sm=tab_opt"
    browser.get(url)
    browser.implicitly_wait(2)

    titles = []
    crwl_blog = []
    # 스크롤 횟수 
    print('스크롤 횟수 : 30')
    for i in range(30):
        browser.find_element(By.CSS_SELECTOR,"body").send_keys(Keys.END)
        time.sleep(1)
        browser.execute_script("return window.scrollY")
    links = browser.find_elements(By.CSS_SELECTOR,"div.total_area > a")
    blog_url = []
    for link in links:
        link = link.get_attribute('href')
        if 'blog.naver.com' in link:
            blog_url.append(link)
    
    print('blog url 개수 : ', len(blog_url))
    for i in range(len(blog_url)):
        blog_text = ''
        browser.get(blog_url[i])
        browser.implicitly_wait(2.5)
        browser.switch_to.frame("mainFrame")
        soup = BeautifulSoup(browser.page_source,'html.parser')
        
        # title 긁어 오기
        title = soup.select("div.pcol1")
        for element in soup.select("div.pcol1"):
            text = element.get_text(strip=True)
            titles.append(text)

        # 본문 긁어 오기
        text = soup.select("div.se-main-container")
        #text = soup.select('div.se-component-content') # 이웃추가까지 긁힘
        for t in text:
            blog_text += t.text
        blog_text = re.sub('\n+', ' ', blog_text)
        blog_text = re.sub('\u200b+', ' ', blog_text)
        crwl_blog.append(blog_text)   

    result = pd.DataFrame(zip(titles, crwl_blog), columns = ['title', 'content'])
    browser.close()
    return result

In [11]:
def ToCsv(k, result):
    '''
    검색 결과 CSV 변환
    폴더 있는지 확인 -> 없으면 생성 -> path에 저장하는 기능 추가하기
    '''
    #save_path = Path('crwl_result_E/')
    save_name = k + '.csv'
    pd.DataFrame(result).to_csv(save_name, encoding='UTF-8')

In [14]:
# 검색 리스트
mbti_e = ['ENFP', 'ENFJ', 'ENTP', 'ENTJ',
          'ESFP', 'ESFJ', 'ESTP', 'ESTJ']

for k in mbti_e:
    key = f"{k} 특징"
    crwl_result = Crawl(key)
    ToCsv(k, crwl_result)

스크롤 횟수 : 30
blog url 개수 :  883
스크롤 횟수 : 30
blog url 개수 :  882
스크롤 횟수 : 30
blog url 개수 :  887
스크롤 횟수 : 30
blog url 개수 :  880
스크롤 횟수 : 30
blog url 개수 :  886
스크롤 횟수 : 30
blog url 개수 :  889
스크롤 횟수 : 30
blog url 개수 :  874
스크롤 횟수 : 30
blog url 개수 :  857
